In [24]:
library(tidyverse)
library(survival)
library(survminer)
library(DESeq2)

# Custom package
library(rutils)

In [25]:
dirs <- rutils::get_dev_directories(dev_paths_file = "../dev_paths.txt")
projects <- c("TCGA-CESC", "TCGA-UCS", "TCGA-UCEC", "TCGA-OV")
unified_dsets <- c("unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data")
matrisome_list <- paste0(dirs$data_dir, "/matrisome/matrisome_hs_masterlist.tsv")

In [26]:
matrisome_df <- rutils::load_matrisome_df(matrisome_list)
for (i in seq_len(length(unified_dsets))) {
    counts_df <- read_tsv(paste0(dirs$data_dir, "/", unified_dsets[i], "/counts.tsv")) %>%
        dplyr::rename(geneID = Hugo_Symbol) %>%
        dplyr::select(-Entrez_Gene_Id) %>%
        dplyr::mutate_if(is.numeric, round, 0)
    
    counts <- as.matrix(counts_df[, -1])
    rownames(counts) <- counts_df$geneID
    norm_counts <- varianceStabilizingTransformation(counts)
    
    matrisome_counts_df <- counts_df %>%
        dplyr::filter(geneID %in% matrisome_df$gene_symbol)

    norm_matrisome_counts_df <- norm_counts %>%
        as_tibble(rownames = "geneID") %>%
        dplyr::filter(geneID %in% matrisome_df$gene_symbol)
    
    write_tsv(matrisome_counts_df, paste0(dirs$data_dir, "/", unified_dsets[i], "/matrisome_counts.tsv"))
    write_tsv(norm_matrisome_counts_df, paste0(dirs$data_dir, "/", unified_dsets[i], "/norm_matrisome_counts.tsv"))
}

Parsed with column specification:
cols(
  Division = col_character(),
  Category = col_character(),
  `Gene Symbol` = col_character(),
  `Gene Name` = col_character(),
  Synonyms = col_character(),
  HGNC_IDs = col_double(),
  `HGNC_IDs Links` = col_double(),
  UniProt_IDs = col_character(),
  Refseq_IDs = col_character(),
  Orthology = col_character(),
  Notes = col_character()
)
Parsed with column specification:
cols(
  .default = col_double(),
  Hugo_Symbol = col_character()
)
See spec(...) for full column specifications.
converting counts to integer mode
Parsed with column specification:
cols(
  .default = col_double(),
  Hugo_Symbol = col_character()
)
See spec(...) for full column specifications.
converting counts to integer mode
Parsed with column specification:
cols(
  .default = col_double(),
  Hugo_Symbol = col_character()
)
See spec(...) for full column specifications.
converting counts to integer mode
